In [31]:
#!/usr/bin/env Python
#coding=utf-8
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.MIMEMultipart import MIMEMultipart
import string
from sklearn import preprocessing
import numpy as np
import sys
import os
import codecs
import math
import json
import urllib
import datetime
import time
import re
import string
import pandas.io.sql as psql
import xlrd
from openpyxl import load_workbook
import scipy as sp
import MySQLdb
import psycopg2
import pyhs2
from sqlalchemy import create_engine
from datetime import timedelta
import calendar
import datetime

reload(sys)
sys.setdefaultencoding('utf8')

In [32]:
df_movie = pd.read_csv('/database/liujy/movie_recommend/movies.csv')
df_rating = pd.read_csv('/database/liujy/movie_recommend/ratings.csv')
df_users = pd.read_csv('/database/liujy/movie_recommend/users.csv')

In [33]:
df_users.columns=['UserID','Gender','Age','JobID','Zip-code']

In [4]:
#df_rating['date']=pd.to_datetime(df_rating['Timestamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M')

In [37]:
#df_link=pd.merge(df_movie,df_rating,how='inner',on='MovieID') #电影及其评分
#df_link.Title.value_counts()[:20]#评分次数最多的top20

In [7]:
#df_link1=pd.merge(df_link,df_users,how='inner',on='UserID')

In [10]:
#df_link1[['UserID','Gender']].drop_duplicates()
#df_link2=df_link1.groupby(["Gender"],as_index=False).agg({"UserID":"count"})

In [35]:

df_movie['Title']=df_movie['Title'].astype('str')
#df_users.filter(regex='UserID|Gender|Age|JobID')
#df_users.values

In [36]:
#用户数据
df_users = df_users.filter(regex='UserID|Gender|Age|JobID')
users_orig = df_users.values
#处理用户数据的年龄和性别
gender_map = {'F':0, 'M':1}
df_users['Gender'] = df_users['Gender'].map(gender_map)

age_map = {val:ii for ii,val in enumerate(set(df_users['Age']))}
df_users['Age'] = df_users['Age'].map(age_map)
#读取Movie数据集
movies_orig = df_movie.values
#将Title中的年份去掉
pattern = re.compile(r'^(.*?)\(?(\d*)\)?$')

title_map = {val:pattern.match(val).group(1) for ii,val in enumerate(set(df_movie['Title']))}
df_movie['Title'] = df_movie['Title'].map(title_map)


#电影Title转数字字典
title_set = set()
for val in df_movie['Title'].str.split():
    title_set.update(val)

title_set.add('<PAD>')
title2int = {val:ii for ii, val in enumerate(title_set)}


#将电影Title转成等长数字列表，长度是15
title_count = 15
title_map = {val:[title2int[row] for row in val.split()] for ii,val in enumerate(set(df_movie['Title']))}

for key in title_map:
    for cnt in range(title_count - len(title_map[key])):
        title_map[key].insert(len(title_map[key]) + cnt,title2int['<PAD>'])

df_movie['Title'] = df_movie['Title'].map(title_map)


In [37]:
#电影类型转数字字典
df_movie['Genres']=df_movie['Genres'].astype('str')
genres_set = set()
for val in df_movie['Genres'].str.split('|'):
    genres_set.update(val)

genres_set.add('<PAD>')
genres2int = {val:ii for ii, val in enumerate(genres_set)}

#将电影类型转成等长数字列表，长度是18
genres_map = {val:[genres2int[row] for row in val.split('|')] for ii,val in enumerate(set(df_movie['Genres']))}

for key in genres_map:
    for cnt in range(max(genres2int.values()) - len(genres_map[key])):
        genres_map[key].insert(len(genres_map[key]) + cnt,genres2int['<PAD>'])

df_movie['Genres'] = df_movie['Genres'].map(genres_map)

In [38]:
#合并三个表
data = pd.merge(pd.merge(df_rating, df_users), df_movie)

In [39]:
data.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,JobID,Title,Genres
0,1,1193,5,978300760,0,0,10,"[3747, 3981, 640, 850, 3482, 2210, 2548, 2548,...","[1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
1,2,1193,5,978298413,1,5,16,"[3747, 3981, 640, 850, 3482, 2210, 2548, 2548,...","[1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
2,12,1193,4,978220179,1,6,12,"[3747, 3981, 640, 850, 3482, 2210, 2548, 2548,...","[1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
3,15,1193,4,978199279,1,6,7,"[3747, 3981, 640, 850, 3482, 2210, 2548, 2548,...","[1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
4,17,1193,5,978158471,1,4,1,"[3747, 3981, 640, 850, 3482, 2210, 2548, 2548,...","[1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
